In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import trange

class model(nn.Module):
    def __init__(self,n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=7,stride=1,padding=3, padding_mode='zeros')
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, stride=2)
        self.fc = nn.Linear(in_features=1024, out_features=n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x,kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,kernel_size=2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.avg_pool2d(x,kernel_size=2)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x
    
    
def train(model, train_loader,valid_loader, optimizer, criterion, device,epochs = 10):
    model.train()
    history = {'train':{'loss':[],'accuracy':[]}, 'valid':{'loss':[],'accuracy':[]}}
    n = len(train_loader)
    for epoch in trange(epochs):
        Loss_epoch = 0
        for idx, data in enumerate(train_loader):
            input, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            output = model(input)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            
            # Loss_epoch += loss.item()
        # Loss_history.append(Loss_epoch/n)
        evaluation_train = evaluate(model, train_loader,criterion, device)
        evaluation_valid = evaluate(model, valid_loader,criterion, device)
        history['train']['accuracy'].append(evaluation_train['accuracy'])
        history['train']['loss'].append(evaluation_train['loss'])

        history['valid']['loss'].append(evaluation_valid['loss'])
        history['valid']['accuracy'].append(evaluation_valid['accuracy'])
    return history


def accuracy(output, labels):
    preds = torch.max(output, dim=1)
    return torch.mean(preds == labels).item()

def evaluate(model,data_loader, criterion, device):
    model.eval()
    Accuracy_history = []
    Loss_history = []
    with torch.no_grad():
        for idx, data in enumerate(data_loader):
            input, target = data[0].to(device), data[1].to(device)
            output = model(input)
            loss = criterion(output, target)
            Accuracy_history.append(accuracy(output, target))
            Loss_history.append(loss.item())
    return {'accuracy': torch.mean(Accuracy_history).item(), 'loss': torch.mean(Loss_history).item()}
            
        


In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

model = resnet50(pretrained=True)
target_layers = [model.layer4[-1]]
input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs